In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [217]:
df = pd.read_csv('Data/df18to21_cleaned.csv')

In [218]:
df.head()

,Unnamed: 0,sentence_length,year_sentenced,sentence_type,guideline_range,imprisoned,guideline_var_pct,guideline_var_mth,dependents,count_convictons,race,education,disposition,citizen,state,criminal_hist,drug_type,case_type,age,armed_career,weapon,max_term,min_term,presentence_stat,gender,crime_type,region
0,12990,188,2018,1,1.0,1,NaN,NaN,2.0,1,1,1.0,3,1.0,Utah,1.0,3.0,1.0,24.0,0.0,0,9996.0,120.0,1.0,0.0,10,West
1,14616,0,2018,1,NaN,1,NaN,NaN,NaN,1,1,NaN,1,3.0,Virginia,1.0,NaN,NaN,28.0,NaN,0,NaN,NaN,1.0,0.0,17,South
2,14617,300,2018,1,6.0,1,NaN,NaN,2.0,3,1,1.0,3,1.0,North Dakota,1.0,6.0,1.0,41.0,0.0,1,9996.0,300.0,1.0,0.0,10,Midwest
3,14619,63,2018,1,1.0,1,NaN,NaN,0.0,1,2,1.0,1,1.0,Tennessee,1.0,NaN,1.0,25.0,0.0,1,120.0,0.0,1.0,0.0,26,South
4,14620,0,2018,1,2.0,1,NaN,NaN,NaN,1,1,NaN,1,3.0,Colorado,1.0,NaN,1.0,27.0,0.0,0,24.0,0.0,1.0,0.0,17,West


In [219]:
df.shape

(251583, 27)

In [220]:
df.MONRACE.value_counts(), df.MONSEX.value_counts()

AttributeError: 'DataFrame' object has no attribute 'MONRACE'

In [221]:
df.CITIZEN.value_counts(), df.NEWEDUC.value_counts(), df.REGION.value_counts()

AttributeError: 'DataFrame' object has no attribute 'CITIZEN'

In [222]:
df.MONRACE.isna().sum(), df.MONSEX.isna().sum(), df.AGE.isna().sum(), df.CITIZEN.isna().sum(), df.REGION.isna().sum()

AttributeError: 'DataFrame' object has no attribute 'MONRACE'

In [223]:
df.NUMDEPEN.isna().sum(), df.NEWEDUC.isna().sum()

AttributeError: 'DataFrame' object has no attribute 'NUMDEPEN'

In [204]:
#Dropping rows with NAN in demo features
#Ignoring 'NEWEDUC' and 'NUMDEPEN' for now, both have high numbers of NAN
df_cln = df.dropna(subset= ['MONRACE', 'MONSEX', 'CITIZEN', 'REGION', 'AGE'])  

In [181]:
# Column Transformer to OneHotEncode/Scale demo features
ohe = OneHotEncoder(drop=None, handle_unknown='ignore', sparse=False) 
sc = StandardScaler()
ct = make_column_transformer(
    (ohe, ['MONRACE', 'MONSEX', 'CITIZEN', 'REGION']),   # , 'NEWEDUC'
    (sc, ['AGE']),                                       # , 'NUMDEPEN'
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [182]:

# df_cln_demo = pd.DataFrame(ct.fit_transform(df_cln), columns=ct.get_feature_names_out())
# df_cln_demo

#### Analysis CASETYPE(split into Felony/Misdemeanor), Target is TOTPRISN(months imprisonment)

In [109]:
df_cln.TOTPRISN.isna().sum()

0

In [110]:
(df_cln.TOTPRISN == 0).sum(), (df_cln.TOTPRISN > 0).sum()

(80733, 185247)

In [111]:
df_cln.CASETYPE.value_counts()

1.0    253353
2.0      7904
Name: CASETYPE, dtype: int64

In [112]:
df_felony = df[df['CASETYPE'] == 1]

In [132]:
X = df_cln[['MONRACE', 'MONSEX', 'CITIZEN', 'REGION', 'AGE']]
y = df_cln['TOTPRISN']

In [133]:
X = pd.DataFrame(ct.fit_transform(X), columns=ct.get_feature_names_out())
X

,MONRACE_1,MONRACE_2,MONRACE_3,MONRACE_4,MONRACE_5,MONRACE_7,MONRACE_8,MONRACE_9,MONSEX_0.0,MONSEX_1.0,CITIZEN_1.0,CITIZEN_2.0,CITIZEN_3.0,CITIZEN_4.0,CITIZEN_5.0,REGION_Mid-Atlantic,REGION_Mid-West,REGION_New England,REGION_South,REGION_South-West,REGION_Territories,REGION_West,AGE
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.868002
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.136568
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.684516
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.685353
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.689086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265975,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.775840
265976,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.410542
265977,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.958489
265978,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.502704


In [143]:
lr = LinearRegression()
lr.fit(X, y)
lr.score(X, y)

0.009230170708425689

In [144]:
pd.DataFrame(lr.coef_, X.columns)      

,0
MONRACE_1,-2.066387e+12
MONRACE_2,-2.066387e+12
MONRACE_3,-2.066387e+12
MONRACE_4,-2.066387e+12
MONRACE_5,-2.066387e+12
MONRACE_7,-2.066387e+12
MONRACE_8,-2.066387e+12
MONRACE_9,-2.066387e+12
MONSEX_0.0,1.865277e+11
MONSEX_1.0,1.865277e+11


#### No significant relationship with any feature other than age. Next, variables will be isolated and made binary.

In [149]:
# LR with just age(not scaled)
X = df_cln[['AGE']]
y = df_cln['TOTPRISN']
lr.fit(X, y)
lr.score(X, y)

0.00031757281805566695

In [148]:
lr.coef_

array([0.65782367])

In [166]:
df_cln['MONRACE'].value_counts()

1    193354
2     58977
3      5637
4      4016
8      2562
7      1299
5       116
9        19
Name: MONRACE, dtype: int64

In [215]:
df_cln[df_cln['MONRACE'] == 1]

,Unnamed: 0,TOTPRISN,SENTYR,SENTIMP,RANGEPT,PRISDUM,PCNTDEPT,MNTHDEPT,NUMDEPEN,NOCOUNTS,MONRACE,NEWEDUC,DISPOSIT,CITIZEN,INOUT,DISTRICT,SENTRNGE,CRIMHIST,COMBDRG2,CASETYPE,AGE,ACCAP,WEAPON,STATMAX,STATMIN,PRESENT,MONSEX,OFFGUIDE,REGION
0,0,172,2017,1,NaN,1,8.510638,16.0,2.0,1,1,3.0,1,1.0,NaN,Tennessee,7.0,1.0,6.0,1.0,46.0,0.0,0,480.0,60.0,1.0,0.0,10,South
1,1,0,2017,1,2.0,1,NaN,NaN,1.0,1,1,1.0,1,3.0,0.0,New Mexico,0.0,1.0,NaN,1.0,35.0,0.0,0,24.0,0.0,1.0,1.0,17,South-West
2,2,6,2017,1,NaN,1,40.000000,4.0,1.0,1,1,1.0,1,3.0,NaN,New Mexico,3.0,1.0,NaN,1.0,29.0,0.0,0,120.0,0.0,1.0,0.0,17,South-West
3,3,4,2017,1,NaN,1,50.000000,4.0,2.0,1,1,1.0,1,3.0,0.0,New Mexico,3.0,1.0,NaN,1.0,44.0,0.0,0,120.0,0.0,1.0,0.0,17,South-West
4,4,0,2017,1,2.0,1,NaN,NaN,3.0,1,1,1.0,1,3.0,0.0,New Mexico,0.0,1.0,NaN,1.0,18.0,0.0,0,24.0,0.0,1.0,0.0,17,South-West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267785,267808,0,2021,1,NaN,1,NaN,NaN,NaN,1,1,NaN,1,3.0,NaN,Texas,NaN,1.0,NaN,NaN,29.0,NaN,0,NaN,NaN,NaN,0.0,17,South-West
267786,267809,0,2021,4,NaN,0,100.000000,10.0,0.0,1,1,5.0,1,1.0,NaN,New York,8.0,1.0,4.0,1.0,76.0,0.0,0,60.0,0.0,2.0,0.0,10,Mid-Atlantic
267787,267810,132,2021,1,NaN,1,2.220000,3.0,0.0,2,1,5.0,1,1.0,NaN,New York,8.0,1.0,NaN,1.0,28.0,0.0,1,9996.0,84.0,1.0,0.0,26,Mid-Atlantic
267788,267811,120,2021,1,6.0,1,NaN,NaN,1.0,1,1,1.0,1,1.0,NaN,Puerto Rico,0.0,1.0,1.0,1.0,32.0,0.0,0,9996.0,120.0,1.0,0.0,10,Territories


In [205]:
df_cln.loc[df_cln['MONRACE'] != 1]

,Unnamed: 0,TOTPRISN,SENTYR,SENTIMP,RANGEPT,PRISDUM,PCNTDEPT,MNTHDEPT,NUMDEPEN,NOCOUNTS,MONRACE,NEWEDUC,DISPOSIT,CITIZEN,INOUT,DISTRICT,SENTRNGE,CRIMHIST,COMBDRG2,CASETYPE,AGE,ACCAP,WEAPON,STATMAX,STATMIN,PRESENT,MONSEX,OFFGUIDE,REGION
10,10,24,2017,1,5.0,1,NaN,NaN,2.0,1,2,5.0,1,1.0,NaN,Iowa,0.0,1.0,NaN,1.0,26.0,0.0,0,120.0,0.0,3.0,1.0,13,Mid-West
14,14,60,2017,1,NaN,1,4.761905,3.00,2.0,1,2,1.0,1,1.0,NaN,Tennessee,7.0,1.0,NaN,1.0,27.0,0.0,0,120.0,0.0,1.0,0.0,13,South
19,19,60,2017,1,NaN,1,14.285714,10.00,0.0,1,2,3.0,1,1.0,NaN,Florida,2.0,1.0,NaN,1.0,28.0,0.0,0,120.0,0.0,1.0,0.0,13,South
20,20,9,2017,1,2.0,1,NaN,NaN,2.0,1,3,1.0,1,1.0,0.0,South Dakota,0.0,0.0,NaN,1.0,21.0,0.0,0,240.0,0.0,1.0,0.0,1,Mid-West
21,21,102,2017,1,2.0,1,NaN,NaN,4.0,1,3,1.0,1,1.0,NaN,South Dakota,0.0,1.0,NaN,1.0,29.0,0.0,1,180.0,0.0,1.0,0.0,26,Mid-West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267776,267799,0,2020,4,1.0,0,NaN,NaN,0.0,1,2,5.0,1,1.0,1.0,Georgia,0.0,1.0,NaN,1.0,37.0,0.0,0,60.0,0.0,3.0,1.0,16,South
267777,267800,0,2021,1,NaN,1,99.880000,23.97,1.0,1,2,5.0,1,1.0,NaN,Michigan,2.0,1.0,NaN,1.0,40.0,0.0,0,60.0,0.0,2.0,1.0,16,Mid-West
267784,267807,70,2021,1,1.0,1,NaN,NaN,2.0,1,2,5.0,1,1.0,NaN,Oklahoma,0.0,1.0,NaN,1.0,44.0,0.0,0,120.0,0.0,1.0,0.0,13,South-West
267789,267812,372,2021,1,NaN,1,NaN,NaN,0.0,2,2,1.0,1,1.0,NaN,New York,8.0,1.0,NaN,1.0,26.0,0.0,1,9996.0,180.0,1.0,0.0,22,Mid-Atlantic


In [199]:
# Race binary (white/minority)
df_cln.loc[df_cln['MONRACE'] != 1] = 0
df_cln['MONRACE'].value_counts()

C:\Users\jswil\AppData\Local\Temp/ipykernel_4136/3248573959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cln[df_cln['MONRACE'] != 1] = 0
C:\Users\jswil\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


1    193354
0     72626
Name: MONRACE, dtype: int64

In [168]:
X = df_cln[['MONRACE']]
y = df_cln['TOTPRISN']
lr.fit(X, y)
lr.score(X, y)

0.005451888456741494

In [169]:
lr.coef_

array([45.6014926])

In [170]:
df_cln[df_cln['MONRACE'] == 1].TOTPRISN.mean()

45.601492599066994

In [211]:
df_cln[df_cln['MONRACE'] != 1].TOTPRISN.mean()

91.75584501418224

In [150]:
#df_cln[df_clean['CASETYPE'] == 2] = 0

In [208]:
df[df['MONRACE'] == 1]

,Unnamed: 0,TOTPRISN,SENTYR,SENTIMP,RANGEPT,PRISDUM,PCNTDEPT,MNTHDEPT,NUMDEPEN,NOCOUNTS,MONRACE,NEWEDUC,DISPOSIT,CITIZEN,INOUT,DISTRICT,SENTRNGE,CRIMHIST,COMBDRG2,CASETYPE,AGE,ACCAP,WEAPON,STATMAX,STATMIN,PRESENT,MONSEX,OFFGUIDE,REGION
0,0,172,2017,1,NaN,1,8.510638,16.0,2.0,1,1,3.0,1,1.0,NaN,Tennessee,7.0,1.0,6.0,1.0,46.0,0.0,0,480.0,60.0,1.0,0.0,10,South
1,1,0,2017,1,2.0,1,NaN,NaN,1.0,1,1,1.0,1,3.0,0.0,New Mexico,0.0,1.0,NaN,1.0,35.0,0.0,0,24.0,0.0,1.0,1.0,17,South-West
2,2,6,2017,1,NaN,1,40.000000,4.0,1.0,1,1,1.0,1,3.0,NaN,New Mexico,3.0,1.0,NaN,1.0,29.0,0.0,0,120.0,0.0,1.0,0.0,17,South-West
3,3,4,2017,1,NaN,1,50.000000,4.0,2.0,1,1,1.0,1,3.0,0.0,New Mexico,3.0,1.0,NaN,1.0,44.0,0.0,0,120.0,0.0,1.0,0.0,17,South-West
4,4,0,2017,1,2.0,1,NaN,NaN,3.0,1,1,1.0,1,3.0,0.0,New Mexico,0.0,1.0,NaN,1.0,18.0,0.0,0,24.0,0.0,1.0,0.0,17,South-West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267785,267808,0,2021,1,NaN,1,NaN,NaN,NaN,1,1,NaN,1,3.0,NaN,Texas,NaN,1.0,NaN,NaN,29.0,NaN,0,NaN,NaN,NaN,0.0,17,South-West
267786,267809,0,2021,4,NaN,0,100.000000,10.0,0.0,1,1,5.0,1,1.0,NaN,New York,8.0,1.0,4.0,1.0,76.0,0.0,0,60.0,0.0,2.0,0.0,10,Mid-Atlantic
267787,267810,132,2021,1,NaN,1,2.220000,3.0,0.0,2,1,5.0,1,1.0,NaN,New York,8.0,1.0,NaN,1.0,28.0,0.0,1,9996.0,84.0,1.0,0.0,26,Mid-Atlantic
267788,267811,120,2021,1,6.0,1,NaN,NaN,1.0,1,1,1.0,1,1.0,NaN,Puerto Rico,0.0,1.0,1.0,1.0,32.0,0.0,0,9996.0,120.0,1.0,0.0,10,Territories


In [209]:
df

,Unnamed: 0,TOTPRISN,SENTYR,SENTIMP,RANGEPT,PRISDUM,PCNTDEPT,MNTHDEPT,NUMDEPEN,NOCOUNTS,MONRACE,NEWEDUC,DISPOSIT,CITIZEN,INOUT,DISTRICT,SENTRNGE,CRIMHIST,COMBDRG2,CASETYPE,AGE,ACCAP,WEAPON,STATMAX,STATMIN,PRESENT,MONSEX,OFFGUIDE,REGION
0,0,172,2017,1,NaN,1,8.510638,16.0,2.0,1,1,3.0,1,1.0,NaN,Tennessee,7.0,1.0,6.0,1.0,46.0,0.0,0,480.0,60.0,1.0,0.0,10,South
1,1,0,2017,1,2.0,1,NaN,NaN,1.0,1,1,1.0,1,3.0,0.0,New Mexico,0.0,1.0,NaN,1.0,35.0,0.0,0,24.0,0.0,1.0,1.0,17,South-West
2,2,6,2017,1,NaN,1,40.000000,4.0,1.0,1,1,1.0,1,3.0,NaN,New Mexico,3.0,1.0,NaN,1.0,29.0,0.0,0,120.0,0.0,1.0,0.0,17,South-West
3,3,4,2017,1,NaN,1,50.000000,4.0,2.0,1,1,1.0,1,3.0,0.0,New Mexico,3.0,1.0,NaN,1.0,44.0,0.0,0,120.0,0.0,1.0,0.0,17,South-West
4,4,0,2017,1,2.0,1,NaN,NaN,3.0,1,1,1.0,1,3.0,0.0,New Mexico,0.0,1.0,NaN,1.0,18.0,0.0,0,24.0,0.0,1.0,0.0,17,South-West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267787,267810,132,2021,1,NaN,1,2.220000,3.0,0.0,2,1,5.0,1,1.0,NaN,New York,8.0,1.0,NaN,1.0,28.0,0.0,1,9996.0,84.0,1.0,0.0,26,Mid-Atlantic
267788,267811,120,2021,1,6.0,1,NaN,NaN,1.0,1,1,1.0,1,1.0,NaN,Puerto Rico,0.0,1.0,1.0,1.0,32.0,0.0,0,9996.0,120.0,1.0,0.0,10,Territories
267789,267812,372,2021,1,NaN,1,NaN,NaN,0.0,2,2,1.0,1,1.0,NaN,New York,8.0,1.0,NaN,1.0,26.0,0.0,1,9996.0,180.0,1.0,0.0,22,Mid-Atlantic
267790,267813,216,2021,2,NaN,1,5.530000,13.0,0.0,1,1,1.0,1,1.0,NaN,Montana,8.0,1.0,NaN,1.0,42.0,0.0,0,360.0,180.0,1.0,0.0,27,West


In [213]:
df['REGION'].value_counts()

South-West      105680
South            63015
West             37206
Mid-West         30390
Mid-Atlantic     21728
New England       5583
Territories       4190
Name: REGION, dtype: int64